<a href="https://colab.research.google.com/github/sanjanasrinivas22/BIS/blob/main/Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import math

weights = np.array([12, 7, 11, 8, 9])
values = np.array([24, 13, 23, 15, 16])
capacity = 26
n = len(weights)

def fitness(sol):
    w = np.sum(sol * weights)
    v = np.sum(sol * values)
    return v if w <= capacity else 0

def levy_flight(l=1.5, d=n):
    a = (math.gamma(1+l) * math.sin(math.pi*l/2) /
         (math.gamma((1+l)/2) * l * 2**((l-1)/2))) ** (1/l)
    u = np.random.normal(0, a, d)
    v = np.random.normal(0, 1, d)
    return u / (np.abs(v) ** (1/l))

def sig(x):
    return 1 / (1 + np.exp(-x))

def cuckoo(pop=20, pa=0.25, it=300):
    nests = np.random.randint(0, 2, (pop, n))
    fit_vals = np.array([fitness(s) for s in nests])

    best_idx = np.argmax(fit_vals)
    best_sol = nests[best_idx].copy()
    best_fit = fit_vals[best_idx]

    for _ in range(it):
        for i in range(pop):
            step = levy_flight()
            y = nests[i] + step
            p = sig(y)
            new = (np.random.rand(n) < p).astype(int)

            f = fitness(new)
            if f > fit_vals[i]:
                nests[i] = new
                fit_vals[i] = f
                if f > best_fit:
                    best_fit = f
                    best_sol = new.copy()

        idx = np.where(np.random.rand(pop) < pa)[0]
        for j in idx:
            nests[j] = np.random.randint(0, 2, n)
            fit_vals[j] = fitness(nests[j])
            if fit_vals[j] > best_fit:
                best_fit = fit_vals[j]
                best_sol = nests[j].copy()

    return best_sol, best_fit

sol, val = cuckoo()
print("Best solution:", sol)
print("Total value:", val)
print("Total weight:", np.sum(sol * weights))


Best solution: [0 1 1 1 0]
Total value: 51
Total weight: 26
